# How will the internet work on Mars: Calculations

**Author:** [David Mytton](https://davidmytton.blog/start), [Console](https://console.dev).

**Correspondence:** <david@console.dev>.

There are many challenges to building human settlements on Mars. The most
efficient launch opportunity windows only arise every 2.2 years when Mars is
closest to Earth. Best journey times are 3-6 months. The atmosphere is
primarily CO2, and it is very cold.

Once we have figured out how to get there and how to reliably support human
life (some are aiming for this decade), questions of quality of life become
relevant. This includes producing water and food, staying fit and healthy,
socialising and entertainment. Normal things humans like to do.

One of the resources we have come to take for granted is access to the
internet. Whether to look up information, send email or watch a video, internet
access is now fundamental to modern life. However, all of these services are
based on Earth. The internet was designed based on a number of assumptions that
will no longer be true if we want to offer the same experience to citizens of
Mars.

In this article we examine those assumptions, discuss the challenges and
consider possible solutions to setting up the internet on Mars. [Read on the
Console website](https://console.dev/research/mars/).

## This notebook

This notebook walks through the calculations used in 
[the article](https://console.dev/research/mars/).

## License

[CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/) i.e. feel free to use
the material and calculations but please include a credit back to
[Console](https://console.dev).

#### Imports

In [ ]:
%pip install -r requirements.txt

import pandas as pd
import pint
from pint import UnitRegistry

ureg = UnitRegistry()

## Problems of distance

If we assume any data is transmitted at the speed of light directly between the planets, the [large (and varying) distance](https://en.wikipedia.org/wiki/Mars#Closest_approaches) means a single data packet can take a prohibitively long time to reach its destination.

In [ ]:
# Calculate the speed of light in a vacuum
lightDistance = 299792458 * ureg.meters
lightTime = 1 * ureg.second
c = lightDistance / lightTime # Speed of light

# Distance from Earth to Mars
# Source: https://en.wikipedia.org/wiki/Mars#Closest_approaches
marsDistance = pd.Series([
    54600000 * ureg.kilometer,
    401000000 * ureg.kilometer
])

# Calculate the range of transmission times
marsTransmissionFastest = marsDistance.min() / c
marsTransmissionMean = marsDistance.values.mean() / c
marsTransmissionSlowest = marsDistance.max() / c

marsTransmission = pd.DataFrame({
    'One way': {
        'fastest': marsDistance.min() / c,
        'average': marsDistance.values.mean() / c,
        'slowest': marsDistance.max() / c,
    },
    'TCP Handshake': {
        'fastest': (marsDistance.min() / c) * 3,
        'average': (marsDistance.values.mean() / c) * 3,
        'slowest': (marsDistance.max() / c) * 3,
    }
})

# Display it nicely
marsTransmissionDisplay = pd.DataFrame({
    'One way': {
        'fastest': f'{marsTransmission["One way"]["fastest"].to("minutes"):.3~}',
        'average': f'{marsTransmission["One way"]["average"].to("minutes"):.3~}',
        'slowest': f'{marsTransmission["One way"]["slowest"].to("minutes"):.3~}',
    },
    'TCP Handshake': {
        'fastest': (f'{marsTransmission["TCP Handshake"]["fastest"].to("minutes"):.3~}'),
        'average': (f'{marsTransmission["TCP Handshake"]["average"].to("minutes"):.3~}'),
        'slowest': (f'{marsTransmission["TCP Handshake"]["slowest"].to("minutes"):.3~}'),
    }
})

marsTransmissionDisplay

,One way,TCP Handshake
fastest,3.04 min,9.11 min
average,12.7 min,38.0 min
slowest,22.3 min,66.9 min


## Sending large files to Mars

Rather than connecting the planets directly, an alternative approach would be to periodically package up the content generated on Earth and physically ship it on the regular missions planned by the likes of SpaceX. Data would be generated on both planets, then asynchronously synced up later.

This might look something like the [AWS Snowmobile](https://aws.amazon.com/snowmobile/) which allows transferring up to 100PB of data inside a shipping container.

### Storage costs

At a list price of US$0.005/GB/month and 100PB maximum capacity, how much would the storage cost be given a 6 month journey time?

In [ ]:
# Probably a way to create a custom pint unit for this calculation
# Feel free to submit a PR if you have time to implement it!
storageCapacity = 100 * ureg.petabytes
journeyTime = 6 * ureg.months
storageCost = 0.005 * storageCapacity.to('gigabytes').magnitude * journeyTime.to('months').magnitude

print(f'Storage cost: US${storageCost:,.0f}')

Storage cost: US$3,000,000


### Shipping cost

[Elon Musk's aspirational goal](https://www.commerce.senate.gov/2004/5/space-shuttle-and-the-future-of-space-launch-vehicles) is US$500 per pound of payload delivered to Earth orbit, so what is the minimum launch cost?

In [ ]:
costPerPound = 500
snowmobileWeight = 68000 # Source: https://aws.amazon.com/snowmobile/faqs/
shippingCost = 500 * snowmobileWeight

print(f'Storage cost: US${shippingCost:,.0f}')

Storage cost: US$34,000,000


The problem is that this is just to Earth orbit and the Falcon Heavy maximum Mars launch capability (37,040 lbs / 16,800 kg) is much less than the weight of the AWS Snowmobile device.  This might mean using AWS Snowball devices instead, which in storage optimized mode can transport up to 80TB each. These [weigh 49.7 lbs (22.54 kg) each](https://docs.aws.amazon.com/snowball/latest/developer-guide/specifications.html).

What could a single Falcon Heavy send to Mars?

In [ ]:
falconHeavyMarsCapacity = 37040 * ureg.lbs
awsSnowballWeight = 49.7 * ureg.lbs
awsSnowballCapacity = 80 * ureg.terabyte
awsSnowballPerFalconHeavy = falconHeavyMarsCapacity / awsSnowballWeight
capacityPerFalconHeavy = awsSnowballPerFalconHeavy * awsSnowballCapacity

print(f'{awsSnowballPerFalconHeavy.magnitude:.0f} AWS Snowball devices carrying a total of {capacityPerFalconHeavy.to("petabyte"):,.0f~} of data.')

745 AWS Snowball devices carrying a total of 60 PB of data.


## Sending YouTube to Mars

Back in 2015, [YouTube was said to generate around 35PB of data per year](https://www.quora.com/What-is-the-total-capacity-of-YouTube-storage/answer/Rasty-Turek), and had at least 400PB of total storage requirements. If that had just continued to grow at the same rate, the total storage for YouTube would now have reached over 600PB.

What would it take to send all of YouTube to Mars?

In [ ]:
youtubeTotal = 600 * ureg.petabytes

# What is the storage cost?
awsSnowballDailyCost = 50 # US$ from # https://aws.amazon.com/snowball/pricing/
# 1 or 3 yr commit discounts are available, which would probably be a good idea
journeyCostPerSnowmobile = journeyTime.to('days').magnitude * awsSnowballDailyCost
storageCostPerFalconHeavy = journeyCostPerSnowmobile * awsSnowballPerFalconHeavy

# How many Falcon Heavy launches would be needed?
falconHeavyCount = youtubeTotal.magnitude / capacityPerFalconHeavy.to('petabyte').magnitude
falconHeavyUnitCost = 94000000 # Launch cost to Mars $94m assuming all stages recoverable

storageCost = falconHeavyCount * storageCostPerFalconHeavy
shippingCost = falconHeavyCount * falconHeavyUnitCost

print(f'Storage cost: US${storageCost.magnitude:,.0f}')
print(f'Shipping cost: US${shippingCost:,.0f} for {falconHeavyCount:.0f} Falcon Heavy launches')

totalCost = storageCost + shippingCost
print(f'Total cost: US${totalCost.magnitude:,.0f}')

Storage cost: US$68,484,375
Shipping cost: US$945,963,823 for 10 Falcon Heavy launches
Total cost: US$1,014,448,198


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c24d71c-4d5b-49bc-a83f-df669e53db37' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>